# Monte Carlo Tree Search Lab

In this lab, we'll be using the game connect four, as a vehicle for learning MinMax and Monte Carlo Tree Search.
We'll also introduce concepts, such as state, that'll stay relevant throughout the course.
Expect to lose in connect four to the algorithm at the end of the lab.

## Setup
This section you won't need to edit, but it is worth skimming through—this is where we declare the objects you'll be interacting with througout the lab.

In [1]:
# imports
import random
from typing import List, Tuple
import time
from copy import deepcopy # world -> thought

In [2]:
# world and world model
class State:
    def __init__(self, cols=7, rows=6, win_req=4):
        self.board = [['.'] * cols for _ in range(rows)]
        self.heights = [1] * cols
        self.num_moves = 0
        self.win_req = win_req

    def get_avail_actions(self) -> List[int]:
        return [i for i in range(len(self.board[0])) if self.heights[i] <= len(self.board)]
  
    def put_action(self, action, agent):
        self.board[len(self.board) - self.heights[action]][action] = agent.name
        self.heights[action] += 1
        self.num_moves += 1

    def is_over(self):
        return self.num_moves >= len(self.board) * len(self.board[0])

    def __repr__(self):
        return self.__str__()
    
    def __str__(self):
        header  = " ".join([str(i) for i in range(len(self.board[0]))])
        line    = "".join(["-" for _ in range(len(header))])
        board   = [[e for e in row] for row in self.board]
        board   = '\n'.join([' '.join(row) for row in board])
        return  '\n' + header + '\n' + line + '\n' + board + '\n'


In [3]:
# evaluate the utility of a state
def utility(state: 'State'):
    board = state.board
    n_cols = len(board[0]) - 1
    n_rows = len(board) - 1

    def diags_pos():
        """Get positive diagonals, going from bottom-left to top-right."""
        for di in ([(j, i - j) for j in range(n_cols)] for i in range(n_cols + n_rows - 1)):
            yield [board[i][j] for i, j in di if i >= 0 and j >= 0 and i < n_cols and j < n_rows]

    def diags_neg():
        """Get negative diagonals, going from top-left to bottom-right."""
        for di in ([(j, i - n_cols + j + 1) for j in range(n_cols)] for i in range(n_cols + n_rows - 1)):
            yield [board[i][j] for i, j in di if i >= 0 and j >= 0 and i < n_cols and j < n_rows]

    cols = list(map(list, list(zip(*board))))
    rows = board
    diags = list(diags_neg()) + list(diags_pos())
    lines = rows + cols + diags
    strings = ["".join(s) for s in lines]
    for string in strings:
        if 'OOOO' in string:
            return -1
        if 'XXXX' in string:
            return 1
    return 0


In [4]:
# parrent class for mcts, minmax, human, and any other idea for an agent you have
class Agent:
    def __init__(self, name: str):
        self.name: str = name

    def get_action(self, state: State):
        return random.choice(state.get_avail_actions())

In [5]:
# connecting states and agents
class Game:
    def __init__(self, agents: Tuple[Agent]):
        self.agents = agents
        self.state = State()

    def play(self):
        while utility(self.state) == 0 and not self.state.is_over():
            for agent in agents:
                if utility(self.state) == 0 and not self.state.is_over():
                    action = agent.get_action(self.state)
                    self.state.put_action(action, agent)
                    print(self.state)

## Exercise 0: Discuss and Run game
Let's discuss if the `utility` function best belongs to the state or the agent.
Put the state, agent and game class together so that a game is run.

In [6]:
agents = (Agent('O'), Agent('X'))
game = Game(agents)
game.play()


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . O . . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
X . O . . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . O . . . .
X . O . . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . O . . . .
X X O . . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . O . . . .
X X O O . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . O . . . .
X X O O . X .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. O O . . . .
X X O O . X .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . X . . . .
. O O . . . .
X X O O . X .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . X . . . .
O O

## Exercise 1: Human Agent
Make a child class of `Agent` called `Human`, with the `give_action` method overwritten to take input from you. *hint*: use `int(input())`

In [7]:
class Human:
    def __init__(self, name):
        self.name = name

    def get_action(self, state: State):
        actiontoTake = -1

        human = int(input())
        actions = state.get_avail_actions()
        if human in actions:
            return human
        else:
            actiontoTake = random.choice(state.get_avail_actions())


In [8]:
agents = (Agent('O'), Human('X'))
game = Game(agents)
game.play()



0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . O . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. X . . O . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . O . .
. X . . O . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . O . .
. X X . O . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . O . .
. X X . O . O


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. X . . O . .
. X X . O . O


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. X . . O . O
. X X . O . O


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. X . . . . .
. X . . O . O
. X X . O . O


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. X . . . . .
. X

## Exercise 2: Gekko
Make a child class of `Agent` called `Gekko`, with the `give_action` that is as short sighted as you can possibly make it. Here you'll need to edit both `give_action`, `utility`, and perhaps some helper functions.

In [16]:
class Gekko(Agent):
    def __init__(self, name):
        self.name = name

    def get_action(self, state: State):
        actions = state.get_avail_actions()
        i = 0  


        while i < len(state.get_avail_actions()):
            if state.heights[i] <= 4:
                return actions[i]
            else:
                i += 1

# I have choosen to make an AI that just pusts its tokens in the same place, 
# if there is space. If not goes to the next column 

In [17]:
agents = (Agent('O'), Gekko('X'))
game = Game(agents)
game.play()



0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. O . . . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
X O . . . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. O . . . . .
X O . . . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
X O . . . . .
X O . . . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. O . . . . .
X O . . . . .
X O . . . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
X O . . . . .
X O . . . . .
X O . . . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. O . . . . .
X O . . . . .
X O . . . . .
X O . . . . .



## *Optional exercise: MinMax*
Make a MinMax class:

In [11]:
""" 
class MinMax(Agent):
   def __init__(self, name):
       self.name = name

   def minValue(self, state: State):
      if(state.is_over):
         return utility(state)
      
      v = float('inf')

      for a in state.get_avail_actions:
         v = max(maxValue(v, state.put_action(a, MinMax('O'))))
         return v
      
   def maxValue(self, state: State):
      if(state.is_over):
         return utility(state)

      v = float('-inf')

      for a in state.get_avail_actions:
         v = min(maxValue(v, state.put_action(a, MinMax('X'))))
         return v

   def minmaxValue(self, state:State):
         return minValue(self, state)

   def get_action(self, state: State):
      return minmaxValue(self,state)
"""


" \nclass MinMax(Agent):\n   def __init__(self, name):\n       self.name = name\n\n   def minValue(self, state: State):\n      if(state.is_over):\n         return utility(state)\n      \n      v = float('inf')\n\n      for a in state.get_avail_actions:\n         v = max(maxValue(v, state.put_action(a, MinMax('O'))))\n         return v\n      \n   def maxValue(self, state: State):\n      if(state.is_over):\n         return utility(state)\n\n      v = float('-inf')\n\n      for a in state.get_avail_actions:\n         v = min(maxValue(v, state.put_action(a, MinMax('X'))))\n         return v\n\n   def minmaxValue(self, state:State):\n         return minValue(self, state)\n\n   def get_action(self, state: State):\n      return minmaxValue(self,state)\n"

## Exercise 3: MCTS
Same but for Monte Carlo Tree Search. See if you can beat it.

In [12]:
import math


class Node:
    def __init__(self, state: State, parent):
        #self.children: List['Nodes'] = []
        self.children = []
        self.parent = parent
        self.state: State = state
        self.action = None

        self.q = 0
        self.wins = 0
        self.lost = 0
        self.visits = 0

    def nonTerminal(self):
        actions = len(self.state.get_avail_actions())
        return (actions > 0)

    def notFullyExpanded(self):
        actions = len(self.state.get_avail_actions())
        return ((len(self.children)) < (actions-1))

    def bestChild(self):  # where to increase win ?
        if(self.parent != None):
            self.q = (self.wins/self.visits) + 0.001 * \
                math.sqrt((2*math.log(self.parent.visits))/self.visits)

        self.children.sort(key=lambda x: x.q)

        """ 
        print("wins ", self.wins, " lost ",
              self.lost, "  visits ", self.visits)
        if(self.parent != None):
            print("  prerent visits ", self.parent.visits)
        
        
        print(" best child q",self.children[0].q)
        """
        return self.children[0]


In [13]:

class MCTS(Agent):
    def __init__(self, name):
        super(MCTS, self).__init__(name)
        self.state = State()
        self.player = -1
        self.v_0 = Node(None, None)
        self.graph = None

    def get_action(self, state: State):
        self.state = state
        action = self.search()
        return action

    def getPlayerName(self):
        if (self.player == 1):
            return 'X'
        else:
            return 'O'

    def search(self):
        if(self.v_0.state == None):
            state_init = deepcopy(self.state)
            self.v_0 = Node(state_init, None)

        serach_depth = 5000
        counter = 0

        # and not self.state.is_over()) #and (counter < serach_depth):
        while (utility(self.state) == 0 and not self.state.is_over() and (counter < serach_depth)):
            v_1 = self.treePolicy(self.v_0)
            stateThink = deepcopy(self.state)

            delta = self.defaultpolicy(stateThink)
            self.player *= -1
            self.backup(v_1, delta)
            counter += 1

        return self.v_0.bestChild().action

    def treePolicy(self, node: Node):
        v = node

        while v.nonTerminal():
            if node.notFullyExpanded():
                v = self.expand(node, self.state)
            else:
                if(utility(v.state) > 0):
                    v.wins += 1
                v = node.bestChild()
                break
        return v

    def expand(self, node: Node, state: State):
        actions = state.get_avail_actions()
        action = actions.pop(0)

        stateThink = deepcopy(self.state)
        stateThink.put_action(action, Agent(self.getPlayerName()))

        v_ = Node(stateThink, node)
        v_.action = action
        node.children.append(v_)
        self.player *= -1

        return v_

    def defaultpolicy(self, state: State):
        actions = state.get_avail_actions()
        random.shuffle(actions)
        stateThink = None

        while (len(actions) > 1):
            action = actions.pop(random.randrange(0, len(actions)))
            stateThink = deepcopy(self.state)
            self.player *= -1
            stateThink.put_action(action, Agent(self.getPlayerName()))

        return utility(stateThink)

    def backup(self, node: Node, delta):
        v = node

        while (v.parent != None):
            N_v = v.parent
            N_v.visits += 1
            N_v.q = utility(N_v.state) + delta
            v = v.parent


In [14]:
agents = (Agent('O'), MCTS('X'))
game = Game(agents)
game.play()



0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . O . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
X . . O . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
O . . . . . .
X . . O . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
X . . . . . .
O . . . . . .
X . . O . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
X . . . . . .
O . . . . . .
X . . O O . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
X . . . . . .
X . . . . . .
O . . . . . .
X . . O O . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
X . . . . . .
X . . . . . .
O . . . . . .
X . . O O O .


0 1 2 3 4 5 6
-------------
. . . . . . .
X . . . . . .
X . . . . . .
X . . . . . .
O . . . . . .
X . . O O O .


0 1 2 3 4 5 6
-------------
. . . . . . .
X . . . . . .
X . . . . . .
X . . . . . .
O .

## *Optional exercise: Dynamic Programming*
Then use dynamic programming to make your AI more efficient. You can use the class below (or not)

In [15]:
class TranspositionTable:
    def __init__(self, size=1_000_000):
        self.size = size
        self.vals = [None] * size

    def board_str(self, state: State):
        return ''.join([''.join(c) for c in state.board])

    def put(self, state: State, utility: float):
        bstr = self.board_str(state)
        idx = hash(bstr) % self.size
        self.vals[idx] = (bstr, utility)

    def get(self, state: State):
        bstr = self.board_str(state)
        idx = hash(bstr) % self.size
        stored = self.vals[idx]
        if stored is None:
            return None
        if stored[0] == bstr:
            return stored[1]
        else:
            return None